In [1]:
text = """
artificial intelligence is transforming modern society
it is used in healthcare finance education and transportation
machine learning allows systems to improve automatically with experience
transformer models changed the field of nlp
attention allows the model to focus on relevant context
"""


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D


In [3]:
tokens = text.lower().split()

vocab = sorted(set(tokens))
word_to_idx = {w: i for i, w in enumerate(vocab)}
idx_to_word = {i: w for w, i in word_to_idx.items()}

vocab_size = len(vocab)


In [4]:
seq_length = 5
X = []
y = []

for i in range(len(tokens) - seq_length):
    X.append([word_to_idx[w] for w in tokens[i:i+seq_length]])
    y.append(word_to_idx[tokens[i+seq_length]])

X = np.array(X)
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)


In [5]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim):
        super().__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim
        )
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim)
        ])
        self.norm1 = tf.keras.layers.LayerNormalization()
        self.norm2 = tf.keras.layers.LayerNormalization()

    def call(self, inputs):
        attn_output = self.attention(inputs, inputs)
        out1 = self.norm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        return self.norm2(out1 + ffn_output)


In [6]:
embed_dim = 32

inputs = tf.keras.Input(shape=(seq_length,))
embedding = Embedding(vocab_size, embed_dim)(inputs)

transformer = TransformerBlock(
    embed_dim=embed_dim,
    num_heads=2,
    ff_dim=64
)(embedding)

x = GlobalAveragePooling1D()(transformer)
outputs = Dense(vocab_size, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)


In [7]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy"
)

model.fit(X, y, epochs=50, batch_size=8)


Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 3.7731
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.5100
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.2196 
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 3.1160 
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 2.9459
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.8001 
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.7076 
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.5037 
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.4480 
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 2.3160 
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 2.2190
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 2.0859
Epoch 13/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2.0675
Epoch 14/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.8840
Epoch 15/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.9052
Epoch 16/50
5/5 ━━━━━━━━━━━━━━━━━━

In [8]:
def generate_text(seed_text, steps=20):
    words = seed_text.lower().split()

    for _ in range(steps):
        seq = words[-seq_length:]
        seq_idx = [word_to_idx[w] for w in seq]
        seq_idx = np.array(seq_idx).reshape(1, seq_length)

        prediction = model.predict(seq_idx, verbose=0)
        next_word = idx_to_word[np.argmax(prediction)]

        words.append(next_word)

    return " ".join(words)


In [9]:
seed = "artificial intelligence is transforming modern"
print(generate_text(seed))


artificial intelligence is transforming modern society it used in in healthcare finance education transportation and machine learning allows systems systems to automatically with experience transformer
